In [2]:
from __future__ import division
#支持精确算法

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
#导入pytorch

import numpy as np
#from util import * 似乎被取消了，线不管，用到再说

In [4]:
#获得测试的输入
def get_test_input():
    img=cv2.imread("dog-cycle-car.png")
    img=cv2.resize(img,(416,416))#调整输入大小
    img_=img[:,:,::-1].transpose((2,0,1))
    #img是【h，w，channel】，这里的img[:,:,::-1]是将第三维度channel从opencv的BGR转化为pytorch的RGB，然后transpose（（2，0，1））的意思是将[height,width,channel]->[channel,height,width]
    img_=img_[np.newaxis,:,:,:]/255.0
    #添加一个0维，并进行归一化处理(Add a channel at 0 (for batch) | Normalise)
    img_=torch.from_numpy(img_).float()#转换为float型
    img_=Variable(img_)#转换成变量
    return img_

In [5]:
def parse_cfg(cfgfile):
    """
    输入：配置文件路径
    返回值：列表对象，其中每一个元素为一个字典类型对应于一个要建立的神经网络模块（层）
    """
    # 加载文件并过滤掉文件的多余内容
    file=open(cfgfile,'r')
    lines=file.read().spilt('\n')#store the lines in a list等价于readlines
    lines=[x for x in lines if len(x)>0] #去掉空行
    lines=[x for x in lines if x[0]!='#'] #去掉以#开头的注释行
    lines = [x.rstrip().lstrip() for x in lines]# 去掉左右两边的空格(rstricp是去掉右边的空格，lstrip是去掉左边的空格)
    #cfg文件中的每个块用[]括起来最后组成一个列表，一个block储存一个块的内容，即每个层用一个字典block储存。
    block={}
    blocks=[]
    
    for line in lines:
        if line[0]=="[": #这是cfg文件中一个层（块）的开始
            if len(block) !=0: #如果块内已经存在信息，说明是上一个块的信息还没有保存
                blocks.append(block)#那么这个块（字典）加入到blocks列表中去
                block={}#覆盖掉已储存的block，新建一个空白的块储存描述下一个块的信息（block是字典）
            block["type"]=line[1:-1].rstrip()#把cfg的[]中的块名作为键type的值
        else:
            key,value=line.split("=")#按等号分割
            block[key.rstrip()]=value.lstrip()#左边是key(去掉右空格)，右边是value(去掉左空格)，形成一个block字典的键值对
    blocks.append(block)#退出循环，将最后一个未加入的block加进去
    # print('\n\n'.join([repr(x) for x in blocks]))
    return blocks
                
# 配置文件定义了6种不同type
# 'net': 相当于超参数,网络全局配置的相关参数
# {'convolutional', 'net', 'route', 'shortcut', 'upsample', 'yolo'}

# cfg = parse_cfg("cfg/yolov3.cfg")
# print(cfg)            
                

In [8]:
class EmptyLayer(nn.Module):
    """
    为shortcut layer / route layer准备，具体功能不在此实现，在Darknet类的forward函数中有体现
    """
    def __init__(self):
        super(EmptyLayer,self).__init__()

In [11]:
class DetectionLayer(nn.Module):
    """
    yolo 检测层的具体实现, 在特征图上使用锚点预测目标区域和类别, 功能函数在predict_transform中
    """
    def __init__(self,anchors):
        super(DetectionLayer,self).__init__()
        self.anchors=anchors